Train an ensemble of 50 NN members to predict dynamical regimes from inputs prepared in *data_preparation.ipynb*

This notebook can take a while to run, leaving the training vulnerable to shutdowns and whatnot. As such a python script version of this notebook is provided in ensemble_training.py.

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap, ListedColormap
import xarray as xr
import cartopy.crs as ccrs
import cartopy.feature as cfeature

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import ConfusionMatrixDisplay

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Reshape, Flatten, Input, Dropout, BatchNormalization, Concatenate
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow import keras 

from pickle import dump

In [10]:
missingdataindex = np.load('Data_Step2/missingdataindex_mass_transport.npy')
maskTraining = np.load('Data_Step2/maskTraining_mass_transport.npy')
maskVal = np.load('Data_Step2/maskVal_mass_transport.npy')
maskTest = np.load('Data_Step2/maskTest_mass_transport.npy')

In [11]:
num_clusters = 6

total_features = np.load('Data_Step2/total_features_mass_transport.npy')
total_labels = np.load(f'Data_Step2/total_labels_{num_clusters}_clusters_mass_transport_probs.npy')

train_features = np.load('Data_Step2/train_features_mass_transport.npy')
train_labels = np.load(f'Data_Step2/train_labels_{num_clusters}_clusters_mass_transport_probs.npy')

val_features = np.load('Data_Step2/val_features_mass_transport.npy')
val_labels = np.load(f'Data_Step2/val_labels_{num_clusters}_clusters_mass_transport_probs.npy')

test_features = np.load('Data_Step2/test_features_mass_transport.npy')
test_labels = np.load(f'Data_Step2/test_labels_{num_clusters}_clusters_mass_transport_probs.npy')

In [12]:
scaler = StandardScaler()
scaler.fit(train_features)
scaler.mean_,scaler.scale_

#dump(scaler, open('saved_scalers/scaler.pkl', 'wb'))

(array([-8.78698922e-10,  3.68473252e+03, -2.93517821e-01, -1.78395655e-05,
         7.85068858e-05,  2.14230191e-03,  4.95124218e-07,  6.49555299e-07,
         3.01111445e+08, -1.45646523e+07]),
 array([5.67442693e-09, 1.61048767e+03, 7.99068434e-01, 1.03240757e-04,
        6.63677737e-01, 4.30530251e-02, 1.86624136e-05, 2.45397642e-06,
        1.35950753e+09, 9.36660921e+08]))

In [13]:
X_total_scaled = scaler.transform(total_features)
#Y_total = tf.keras.utils.to_categorical(total_labels)
Y_total = total_labels

X_train_scaled = scaler.transform(train_features)
#Y_train = tf.keras.utils.to_categorical(train_labels)
Y_train = train_labels

X_val_scaled = scaler.transform(val_features)
#Y_val = tf.keras.utils.to_categorical(val_labels)
Y_val = val_labels

X_test_scaled = scaler.transform(test_features)
#Y_test = tf.keras.utils.to_categorical(test_labels)
Y_test = test_labels

In [14]:
X_train_scaled.shape, Y_train.shape

((481049, 10), (481049, 6))

In [15]:
model_name = 'model_24x2_16x2_tanh_6_clusters_probs'

if not os.path.isdir(f'saved_models/{model_name}/'):
    os.makedirs(f'saved_models/{model_name}/')
    print('Model directory created')
else:
    print('Model directory already exists')

Model directory already exists


In [ ]:
for i in range(50):
    
    print(f'Training model {i}')
    model = None
    tf.keras.backend.clear_session()
    member_name = f'model_{i}'

    inputs = Input(shape=(10,))
    x = Dense(24, activation='tanh')(inputs)
    x = Dense(24, activation='tanh')(x)
    x = Dense(16, activation='tanh')(x)
    x = Dense(16, activation='tanh')(x)
    outputs = Dense(num_clusters, activation='softmax')(x)

    model = Model(inputs, outputs)
    model.compile(loss='categorical_crossentropy', optimizer=Adam(1e-4), metrics=['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(k=2)])
    
    es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=0)
    #lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=5)
    
    history = model.fit(X_train_scaled, Y_train,
                        batch_size=32,
                        epochs=100,
                        verbose=0,
                        validation_data=(X_val_scaled, Y_val),
                        shuffle=True,
                        callbacks=[es])
    
    model.save(f'saved_models/{model_name}/{member_name}.h5')